In [1]:
!pip uninstall -y scikit-image
!pip install scikit-image==0.19.3


Found existing installation: scikit-image 0.25.0
Uninstalling scikit-image-0.25.0:
  Successfully uninstalled scikit-image-0.25.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.9/13.9 MB 100.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 97.3 MB/s eta 0:00:00


### **Mounting the drive**

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


# **Importing the train dataset**

In [ ]:
import os

# Path to the train directory
train_dir = '/content/drive/MyDrive/PlantDataSet/train'

# Retrieve the list of classes
classes = sorted(os.listdir(train_dir))

# Display the order of classes
print("Order of classes in the train folder:")
for idx, class_name in enumerate(classes):
    print(f"{idx + 1}. {class_name}")


Order of classes in the train folder:
1. Bacterial Blight (CBB)
2. Brown Streak Disease (CBSD)
3. Green Mottle (CGM)
4. Healthy
5. Mosaic Disease (CMD)
6. Rice_BrownSpot
7. Rice_Healthy
8. Rice_Hispa
9. Rice_LeafBlast
10. apple_apple scab
11. apple_healthy
12. bacterial spot
13. black rot
14. cedar apple rust
15. cherry (including sour)_healthy
16. cherry (including sour)_powdery mildew
17. corn (maize)
18. corn (maize)_cercospora leaf spot gray leaf spot
19. corn (maize)_common rust
20. corn (maize)_healthy
21. corn (maize)_northern leaf blight
22. early blight
23. grape
24. grape_black rot
25. grape_esca (black measles)
26. grape_healthy
27. grape_leaf blight (isariopsis leaf spot)
28. healthy
29. healthy_healthy
30. late blight
31. orange
32. orange_haunglongbing (citrus greening)
33. peach
34. peach_bacterial spot
35. peach_healthy
36. pepper, bell
37. squash_powdery mildew
38. strawberry_healthy
39. strawberry_leaf scorch
40. tomato_bacterial spot
41. tomato_early blight
42. tomat

# **Importing Necessary Libraries**

In [3]:
import cv2
import numpy as np
import tensorflow as tf
import os
import pandas as pd
import matplotlib.pyplot as plt
from skimage.feature import graycomatrix, graycoprops


# **Defining the paths**

In [ ]:
import os

base_path = '/content/drive/MyDrive/PlantDataSet'
train_dir = os.path.join(base_path, 'train')
test_dir = os.path.join(base_path, 'test')
val_dir = os.path.join(base_path, 'validation')
print(train_dir)


/content/drive/MyDrive/PlantDataSet/train


In [ ]:
import tensorflow as tf

train_dataset = tf.keras.utils.image_dataset_from_directory(train_dir, image_size=(150, 150), batch_size=32)

test_dataset = tf.keras.utils.image_dataset_from_directory(test_dir, image_size=(150, 150), batch_size=32)



Found 38104 files belonging to 49 classes.
Found 5741 files belonging to 44 classes.


In [ ]:
validation_dataset = tf.keras.utils.image_dataset_from_directory(
    validation_dir,
    image_size=(150, 150),
    batch_size=32
)

Found 9458 files belonging to 44 classes.


In [ ]:
dataset = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    image_size=(150, 150),
    batch_size=32
)

# Access class names
print(dataset.class_names)


Found 38104 files belonging to 49 classes.
['Bacterial Blight (CBB)', 'Brown Streak Disease (CBSD)', 'Green Mottle (CGM)', 'Healthy', 'Mosaic Disease (CMD)', 'Rice_BrownSpot', 'Rice_Healthy', 'Rice_Hispa', 'Rice_LeafBlast', 'apple_apple scab', 'apple_healthy', 'bacterial spot', 'black rot', 'cedar apple rust', 'cherry (including sour)_healthy', 'cherry (including sour)_powdery mildew', 'corn (maize)', 'corn (maize)_cercospora leaf spot gray leaf spot', 'corn (maize)_common rust', 'corn (maize)_healthy', 'corn (maize)_northern leaf blight', 'early blight', 'grape', 'grape_black rot', 'grape_esca (black measles)', 'grape_healthy', 'grape_leaf blight (isariopsis leaf spot)', 'healthy', 'healthy_healthy', 'late blight', 'orange', 'orange_haunglongbing (citrus greening)', 'peach', 'peach_bacterial spot', 'peach_healthy', 'pepper, bell', 'squash_powdery mildew', 'strawberry_healthy', 'strawberry_leaf scorch', 'tomato_bacterial spot', 'tomato_early blight', 'tomato_healthy', 'tomato_late blig

# **Image Preprocessing**

In [ ]:
import tensorflow as tf

def preprocess_image(image_path):
    # Load the image from the file path
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)  # Decode JPEG image (use decode_png for PNG images)

    # Resize image
    image = tf.image.resize(image, [150, 150])

    # Normalize image (scale pixel values to [0, 1])
    image = image / 255.0

    # Apply basic image processing techniques
    flipped = tf.image.flip_left_right(image)                      # Flip horizontally
    bright = tf.image.adjust_brightness(image, 0.3)               # Brightness adjustment
    contrast = tf.image.adjust_contrast(image, 2)                 # Contrast adjustment
    rotated = tf.image.rot90(image)                               # Rotate 90 degrees
    cropped = tf.image.central_crop(image, central_fraction=0.8)  # Central crop

    # Advanced image processing using TensorFlow
    def add_random_noise(image):
        noise = tf.random.normal(shape=tf.shape(image), mean=0.0, stddev=0.1, dtype=tf.float32)
        return tf.clip_by_value(image + noise, 0.0, 1.0)

    # Applying advanced techniques
    noisy = add_random_noise(image)  # Add random noise

    # Return all processed versions in a dictionary
    return {
        "original": image,
        "flipped": flipped,
        "bright": bright,
        "contrast": contrast,
        "rotated": rotated,
        "cropped": cropped,
        "noisy": noisy
    }


# **Image Segementation**

In [ ]:
import os
import tensorflow as tf
import cv2
import numpy as np

# Function to load image using OpenCV
def load_image_cv2(image_path):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert from BGR to RGB
    return image

# Example of a segmentation function (to be adapted for each segmentation type)
def threshold_segmentation(image):
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    _, thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)
    return tf.convert_to_tensor(thresh, dtype=tf.float32) / 255.0  # Normalize image

def edge_segmentation(image):
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    edges = cv2.Canny(gray, 100, 200)
    return tf.convert_to_tensor(edges, dtype=tf.float32) / 255.0

def kmeans_segmentation(image):
    # Example k-means segmentation (you may need to modify this)
    image_flat = image.reshape((-1, 3))
    kmeans = cv2.KMeans(n_clusters=3, random_state=0).fit(image_flat)
    segmented = kmeans.cluster_centers_[kmeans.labels_]
    segmented_image = segmented.reshape(image.shape)
    return tf.convert_to_tensor(segmented_image, dtype=tf.float32) / 255.0

def contour_segmentation(image):
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    _, thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contour_image = np.zeros_like(image)
    cv2.drawContours(contour_image, contours, -1, (0, 255, 0), 2)  # Green contours
    return tf.convert_to_tensor(contour_image, dtype=tf.float32) / 255.0

def watershed_segmentation(image):
    # Example of watershed segmentation
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    _, thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)
    sure_bg = cv2.dilate(thresh, None, iterations=3)
    dist_transform = cv2.distanceTransform(thresh, cv2.DIST_L2, 5)
    _, sure_fg = cv2.threshold(dist_transform, 0.7 * dist_transform.max(), 255, 0)
    unknown = cv2.subtract(sure_bg, sure_fg)

    # Markers for watershed
    _, markers = cv2.connectedComponents(sure_fg.astype(np.uint8))
    markers = markers + 1
    markers[unknown == 255] = 0

    watershed = cv2.watershed(image, markers)
    image[watershed == -1] = [255, 0, 0]  # Mark watershed boundaries in red
    return tf.convert_to_tensor(image, dtype=tf.float32) / 255.0

# Function to process and return segmented images as model-friendly inputs
def preprocess_and_get_segmented_images(image_path):
    image = load_image_cv2(image_path)

    # Apply different segmentation techniques
    threshold = threshold_segmentation(image)
    edge = edge_segmentation(image)
    kmeans = kmeans_segmentation(image)
    contour = contour_segmentation(image)
    watershed = watershed_segmentation(image)

    # Create a dictionary to store the processed segmented images
    processed_image = {
        "original": tf.convert_to_tensor(image, dtype=tf.float32) / 255.0,  # Normalize original image
        "threshold": threshold,
        "edge": edge,
        "kmeans": kmeans,
        "contour": contour,
        "watershed": watershed
    }

    return processed_image


# **Feature Extraction**

In [ ]:
import os
import random
import cv2
import numpy as np
import tensorflow as tf
from skimage.feature import greycomatrix, greycoprops
from skimage.color import rgb2gray

# Helper function to load an image using OpenCV
def load_image_cv2(image_path):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert from BGR to RGB
    return image

# Function to extract texture features using GLCM
def extract_texture_features_glcm(image):
    gray = rgb2gray(image)  # Convert to grayscale
    glcm = greycomatrix((gray * 255).astype(np.uint8), [1], [0], symmetric=True, normed=True)

    # Extract GLCM features: contrast, dissimilarity, homogeneity, ASM, energy, correlation
    contrast = greycoprops(glcm, 'contrast')[0, 0]
    dissimilarity = greycoprops(glcm, 'dissimilarity')[0, 0]
    homogeneity = greycoprops(glcm, 'homogeneity')[0, 0]
    asm = greycoprops(glcm, 'ASM')[0, 0]
    energy = greycoprops(glcm, 'energy')[0, 0]
    correlation = greycoprops(glcm, 'correlation')[0, 0]

    return tf.convert_to_tensor([contrast, dissimilarity, homogeneity, asm, energy, correlation], dtype=tf.float32)

# Function to extract color histogram features (RGB)
def extract_color_histogram_features(image):
    # Calculate color histograms for each channel (Red, Green, Blue)
    hist_r = cv2.calcHist([image], [0], None, [256], [0, 256])
    hist_g = cv2.calcHist([image], [1], None, [256], [0, 256])
    hist_b = cv2.calcHist([image], [2], None, [256], [0, 256])

    # Flatten histograms and convert to TensorFlow tensor
    hist_r_flat = tf.convert_to_tensor(hist_r.flatten(), dtype=tf.float32)
    hist_g_flat = tf.convert_to_tensor(hist_g.flatten(), dtype=tf.float32)
    hist_b_flat = tf.convert_to_tensor(hist_b.flatten(), dtype=tf.float32)

    return hist_r_flat, hist_g_flat, hist_b_flat

# Function to extract shape features (area and perimeter) using contours
def extract_shape_features(image):
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    _, thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    areas = [cv2.contourArea(c) for c in contours]
    perimeters = [cv2.arcLength(c, True) for c in contours]

    return tf.convert_to_tensor([np.sum(areas), np.sum(perimeters)], dtype=tf.float32)

# Full feature extraction function for segmented images
def extract_features_from_segmented_image(image):
    texture = extract_texture_features_glcm(image)
    color_hist_r, color_hist_g, color_hist_b = extract_color_histogram_features(image)
    shape = extract_shape_features(image)

    # Combine all features into a single tensor
    features = tf.concat([texture, color_hist_r, color_hist_g, color_hist_b, shape], axis=0)
    return features

# Function to process and return feature vectors as model-friendly inputs (for training)
def process_and_generate_features(image_dir, batch_size=32):
    """
    Selects random images from the dataset, extracts features,
    and returns them as tensors ready for input into a model.
    """
    all_image_paths = [os.path.join(root, file)
                       for root, _, files in os.walk(image_dir)
                       for file in files if file.endswith(('.jpg', '.jpeg', '.png'))]

    # Shuffle and batch the image paths
    random.shuffle(all_image_paths)
    batches = [all_image_paths[i:i + batch_size] for i in range(0, len(all_image_paths), batch_size)]

    for batch in batches:
        images = []
        for image_path in batch:
            image = load_image_cv2(image_path)

            # Extract features from the segmented image
            features = extract_features_from_segmented_image(image)

            # Append the feature tensor to the batch
            images.append(features)

        # Return a batch of feature tensors
        yield tf.stack(images)  # Stack all feature vectors into a single tensor



In [ ]:
import tensorflow as tf

train_dataset = tf.keras.utils.image_dataset_from_directory(train_dir, image_size=(150, 150), batch_size=32)

test_dataset = tf.keras.utils.image_dataset_from_directory(test_dir, image_size=(150, 150), batch_size=32)



Found 38104 files belonging to 49 classes.
Found 5741 files belonging to 44 classes.


In [ ]:
import tensorflow as tf

# Load 50% of the data for training
train_dataset = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    image_size=(150, 150),
    batch_size=32,
    subset='training',         # Indicate we are using the training split
    validation_split=0.5,      # 50% of data will be used for validation
    seed=42                    # Ensure reproducibility
)

# Load the other 50% as validation/test data
validation_dataset = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    image_size=(150, 150),
    batch_size=32,
    subset='validation',       # Indicate we are using the validation split
    validation_split=0.5,      # 50% of data will be used for validation
    seed=42                    # Ensure reproducibility
)

# Display dataset details
print(f"Training dataset: {len(train_dataset)} batches")
print(f"Validation dataset: {len(validation_dataset)} batches")


Found 38104 files belonging to 49 classes.
Using 19052 files for training.
Found 38104 files belonging to 49 classes.
Using 19052 files for validation.
Training dataset: 596 batches
Validation dataset: 596 batches


# **Model Training**

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

# Define paths
train_dir = '/content/drive/MyDrive/PlantDataSet/train'  # Full train folder
test_dir = '/content/drive/MyDrive/PlantDataSet/test'  # Test folder

# Use ImageDataGenerator to preprocess data without splitting the data
train_datagen = ImageDataGenerator(rescale=1./255)  # Normalize pixel values for training data

# Training generator (using the entire data from the train folder)
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    shuffle=True,  # Shuffle the data to avoid model bias
    seed=42
)

# Test data generator (directly from the test folder)
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    shuffle=False,  # No shuffling for test data
    seed=42
)

# Define CNN model
cnn = tf.keras.Sequential([
    tf.keras.Input(shape=(128, 128, 3)),
    tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu'),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPool2D(2, 2),
    tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
    tf.keras.layers.Conv2D(64, 3, activation='relu'),
    tf.keras.layers.MaxPool2D(2, 2),
    tf.keras.layers.Conv2D(128, 3, padding='same', activation='relu'),
    tf.keras.layers.Conv2D(128, 3, activation='relu'),
    tf.keras.layers.MaxPool2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(len(train_generator.class_indices), activation='softmax')
])

# Compile the model
cnn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model for up to 10 epochs
cnn.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=20,
    verbose=1
)
# Save the model
model_save_path = '/content/drive/MyDrive/PlantDataSet/sashanksModel20epochhhha.h5'
cnn.save(model_save_path)
# Evaluate on test data
# test_loss, test_accuracy = cnn.evaluate(test_generator)
# print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")



print("Model saved at:", model_save_path)


Found 38104 images belonging to 49 classes.
Found 5741 images belonging to 44 classes.
Epoch 1/20
1191/1191 ━━━━━━━━━━━━━━━━━━━━ 195s 153ms/step - accuracy: 0.2862 - loss: 2.6697
Epoch 2/20
1191/1191 ━━━━━━━━━━━━━━━━━━━━ 0s 13us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 3/20
1191/1191 ━━━━━━━━━━━━━━━━━━━━ 186s 147ms/step - accuracy: 0.6477 - loss: 1.1063
Epoch 4/20
1191/1191 ━━━━━━━━━━━━━━━━━━━━ 0s 7us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 5/20
1191/1191 ━━━━━━━━━━━━━━━━━━━━ 199s 144ms/step - accuracy: 0.7366 - loss: 0.7914
Epoch 6/20
1191/1191 ━━━━━━━━━━━━━━━━━━━━ 0s 7us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 7/20
1191/1191 ━━━━━━━━━━━━━━━━━━━━ 202s 144ms/step - accuracy: 0.7878 - loss: 0.6191
Epoch 8/20
1191/1191 ━━━━━━━━━━━━━━━━━━━━ 0s 6us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 9/20
1191/1191 ━━━━━━━━━━━━━━━━━━━━ 199s 142ms/step - accuracy: 0.8193 - loss: 0.5088
Epoch 10/20
1191/1191 ━━━━━━━━━━━━━━━━━━━━ 0s 6us/step - accuracy: 0

Model saved at: /content/drive/MyDrive/PlantDataSet/sashanksModel20epochhhha.h5


# **Testing With a Simple Random image**

In [ ]:
# Install necessary packages (if running in Colab, they may already be installed)


# Import necessary libraries
import os
import tensorflow as tf
import numpy as np
import cv2
from skimage.feature import local_binary_pattern
from skimage.color import rgb2gray
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Configuration
MODEL_PATH = '/content/drive/MyDrive/PlantDataSet/sashanksModel20epochhhha.h5'  # Adjust the path to your model file
UPLOAD_FOLDER = '/content/uploads/'

# Load the model
model = tf.keras.models.load_model(MODEL_PATH)

# List of class names (make sure this matches the order of your model's output)
CLASS_NAMES = [
    "Bacterial Blight (CBB)",
    "Brown Streak Disease (CBSD)",
    "Green Mottle (CGM)",
    "Healthy",
    "Mosaic Disease (CMD)",
    "Rice_BrownSpot",
    "Rice_Healthy",
    "Rice_Hispa",
    "Rice_LeafBlast",
    "apple_apple scab",
    "apple_healthy",
    "bacterial spot",
    "black rot",
    "cedar apple rust",
    "cherry (including sour)_healthy",
    "cherry (including sour)_powdery mildew",
    "corn (maize)",
    "corn (maize)_cercospora leaf spot gray leaf spot",
    "corn (maize)_common rust",
    "corn (maize)_healthy",
    "corn (maize)_northern leaf blight",
    "early blight",
    "grape",
    "grape_black rot",
    "grape_esca (black measles)",
    "grape_healthy",
    "grape_leaf blight (isariopsis leaf spot)",
    "healthy",
    "healthy_healthy",
    "late blight",
    "orange",
    "orange_haunglongbing (citrus greening)",
    "peach",
    "peach_bacterial spot",
    "peach_healthy",
    "pepper, bell",
    "squash_powdery mildew",
    "strawberry_healthy",
    "strawberry_leaf scorch",
    "tomato_bacterial spot",
    "tomato_early blight",
    "tomato_healthy",
    "tomato_late blight",
    "tomato_leaf mold",
    "tomato_septoria leaf spot",
    "tomato_spider mites two-spotted spider mite",
    "tomato_target spot",
    "tomato_tomato mosaic virus",
    "tomato_tomato yellow leaf curl virus"
]

# Function to preprocess the image for the model
def preprocess_image(image_path):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert to RGB

    # Resize the image to match the input size expected by the model
    image = tf.image.resize(image, [128, 128])  # Resize to 128x128 to match training input

    image = image / 255.0  # Normalize the pixel values to [0, 1]

    # Expand dimensions to match the model input shape
    return np.expand_dims(image, axis=0)  # This makes the shape (1, 128, 128, 3)

# Function to segment the leaf from the image
def segment_leaf(image):
    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

    # Apply Gaussian blur
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # Apply binary thresholding
    _, thresh = cv2.threshold(blurred, 120, 255, cv2.THRESH_BINARY)

    # Find contours in the binary image
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Create a mask with the contours
    mask = np.zeros_like(gray)
    cv2.drawContours(mask, contours, -1, (255), thickness=cv2.FILLED)

    # Apply the mask to the image
    segmented_image = cv2.bitwise_and(image, image, mask=mask)

    return segmented_image

# Function to extract color features
def extract_color_features(image):
    mean_color = np.mean(image, axis=(0, 1))  # Mean color for each channel (R, G, B)
    std_color = np.std(image, axis=(0, 1))    # Standard deviation of color channels
    return mean_color, std_color

# Function to extract texture features (using Local Binary Patterns)
def extract_texture_features(image):
    gray_image = rgb2gray(image)  # Convert to grayscale
    lbp = local_binary_pattern(gray_image, P=8, R=1, method='uniform')  # Calculate LBP
    texture_features = np.histogram(lbp.ravel(), bins=np.arange(0, 59), range=(0, 58), density=True)[0]
    return texture_features

# Function to extract shape features
def extract_shape_features(image):
    gray_image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    _, thresh = cv2.threshold(gray_image, 120, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Extract area and perimeter as shape features
    areas = [cv2.contourArea(c) for c in contours]
    perimeters = [cv2.arcLength(c, True) for c in contours]
    return np.mean(areas), np.mean(perimeters)

# Function to preprocess the image, extract features, and make a prediction
def preprocess_and_predict(image_path):
    image = cv2.imread(image_path)
    segmented_image = segment_leaf(image)

    # Extract color, texture, and shape features
    color_features = extract_color_features(segmented_image)
    texture_features = extract_texture_features(segmented_image)
    shape_features = extract_shape_features(segmented_image)

    # Combine all features into a single array (flattened)
    all_features = np.concatenate([color_features[0], color_features[1], texture_features, shape_features])

    # Preprocess the image for the model prediction
    processed_image = preprocess_image(image_path)

    # Predict using the model
    prediction = model.predict(processed_image)
    predicted_class_index = np.argmax(prediction, axis=1)[0]

    return predicted_class_index, all_features

# Test with an example image (path to image on Google Drive or uploaded)
image_path = '/content/drive/MyDrive/PlantDataSet/validation/pepper, bell_bacterial spot/01613cd0-d3cd-4e96-945c-a312002037bf___JR_B.Spot 3262.JPG'  # Change this path to an actual image on your Drive

# Get prediction and features
predicted_class_index, features = preprocess_and_predict(image_path)

# Output predicted class name
predicted_class_name = CLASS_NAMES[predicted_class_index]
print(f"Predicted Class: {predicted_class_name}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 270ms/step
Predicted Class: bacterial spot


In [ ]:
def preprocess_and_predict(image_path):
    """
    Preprocesses the image and predicts the class.

    Args:
        image_path (str): Path to the image file.

    Returns:
        predicted_class_index (int): Index of the predicted class.
        confidence (float): Confidence of the prediction.
    """
    # Load the image
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = tf.convert_to_tensor(image, dtype=tf.float32)

    # Preprocess the image (resize and normalize)
    image = tf.image.resize(image, [128, 128])  # Adjust size as per your model
    image = image / 255.0  # Normalize to [0, 1]
    image = tf.expand_dims(image, axis=0)  # Add batch dimension

    # Make prediction using your trained model
    predictions = model.predict(image)

    # Get the class index and confidence
    predicted_class_index = np.argmax(predictions, axis=1)[0]
    confidence = np.max(predictions)

    return predicted_class_index, confidence


# **Loading the model**

In [ ]:
from tensorflow.keras.models import load_model
model = load_model('/content/drive/MyDrive/PlantDataSet/sashanksModel20epochhhha.h5')


# **Testing the model on Test Dataset**

In [ ]:
import os
from tqdm import tqdm
from sklearn.metrics import accuracy_score

# Define CLASS_NAMES with all possible class names
CLASS_NAMES = [
    "Cassava_Bacterial Blight (CBB)",
    "Cassava_Brown Streak Disease (CBSD)",
    "Cassava_Green Mottle (CGM)",
    "Cassava_Healthy",
    "Cassava_Mosaic Disease (CMD)",
    "Rice_BrownSpot",
    "Rice_Healthy",
    "Rice_Hispa",
    "Rice_LeafBlast",
    "apple_apple scab",
    "apple_black rot",
    "apple_cedar apple rust",
    "apple_healthy",
    "cherry (including sour)_healthy",
    "cherry (including sour)_powdery mildew",
    "corn (maize)_cercospora leaf spot gray leaf spot",
    "corn (maize)_common rust",
    "corn (maize)_healthy",
    "corn (maize)_northern leaf blight",
    "grape_black rot",
    "grape_esca (black measles)",
    "grape_healthy",
    "grape_leaf blight (isariopsis leaf spot)",
    "orange_haunglongbing (citrus greening)",
    "peach_bacterial spot",
    "peach_healthy",
    "pepper, bell_bacterial spot",
    "pepper, bell_healthy",
    "potato_early blight",
    "potato_healthy",
    "potato_late blight",
    "squash_powdery mildew",
    "strawberry_healthy",
    "strawberry_leaf scorch",
    "tomato_bacterial spot",
    "tomato_early blight",
    "tomato_healthy",
    "tomato_late blight",
    "tomato_leaf mold",
    "tomato_septoria leaf spot",
    "tomato_spider mites two-spotted spider mite",
    "tomato_target spot",
    "tomato_tomato mosaic virus",
    "tomato_tomato yellow leaf curl virus"
]

# Path to the test dataset
TEST_DATASET_PATH = '/content/drive/MyDrive/PlantDataSet/test'

# Define the classes to exclude from the testing process
train_not_in_test = [
    'peach', 'orange', 'Mosaic Disease (CMD)', 'corn (maize)', 'grape',
    'early blight', 'Healthy', 'late blight', 'black rot', 'pepper, bell',
    'healthy', 'cedar apple rust', 'bacterial spot', 'healthy_healthy',
    'Green Mottle (CGM)', 'Bacterial Blight (CBB)', 'Brown Streak Disease (CBSD)'
]

# Identify the classes present in the test dataset
def get_test_classes(test_path):
    test_classes = [d for d in os.listdir(test_path) if os.path.isdir(os.path.join(test_path, d))]
    return sorted(test_classes)  # Ensure consistent order

# Get the test classes
test_classes = get_test_classes(TEST_DATASET_PATH)
print(f"Test Classes: {test_classes}")

# Filter CLASS_NAMES to include only test dataset classes that are not in train_not_in_test
CLASS_NAMES_TEST = [cls for cls in CLASS_NAMES if any(test_class in cls for test_class in test_classes) and not any(exclude_class in cls for exclude_class in train_not_in_test)]

# Function to get true class label from the image path
def get_true_label(image_path):
    for cls in test_classes:
        if cls in image_path:
            return cls
    return None

# Initialize lists for predictions and true labels
true_labels = []
predicted_labels = []

# Get all test images
test_images = []
for root, _, files in os.walk(TEST_DATASET_PATH):
    for file in files:
        if file.endswith(('.png', '.jpg', '.jpeg')):  # Adjust file extensions if needed
            test_images.append(os.path.join(root, file))

# Display overall progress
with tqdm(total=len(test_images), desc="Processing Test Images", unit="image", ncols=100, dynamic_ncols=True, leave=True) as pbar:
    for image_path in test_images:
        # Get the true label from the path
        true_label = get_true_label(image_path)
        if true_label is None:
            pbar.update(1)
            continue  # Skip if the true label cannot be determined

        # Preprocess and predict (Define preprocess_and_predict elsewhere)
        predicted_class_index, _ = preprocess_and_predict(image_path)

        # Validate predicted class index
        if predicted_class_index < 0 or predicted_class_index >= len(CLASS_NAMES):
            print(f"Invalid predicted class index: {predicted_class_index} for image {image_path}")
            continue  # Skip invalid prediction

        predicted_class_name = CLASS_NAMES[predicted_class_index]

        # Append to lists
        true_labels.append(true_label)
        predicted_labels.append(predicted_class_name)

        # Update progress bar without showing inner logs
        pbar.update(1)

# Map true and predicted labels to the subset of test classes
mapped_true_labels = [cls if cls in CLASS_NAMES_TEST else 'Unknown' for cls in true_labels]
mapped_predicted_labels = [cls if cls in CLASS_NAMES_TEST else 'Unknown' for cls in predicted_labels]

# Calculate accuracy
accuracy = accuracy_score(mapped_true_labels, mapped_predicted_labels)
print(f"\nModel Accuracy on Test Dataset (matching classes): {accuracy * 100:.2f}%")


Test Classes: ['Cassava_Bacterial Blight (CBB)', 'Cassava_Brown Streak Disease (CBSD)', 'Cassava_Green Mottle (CGM)', 'Cassava_Healthy', 'Cassava_Mosaic Disease (CMD)', 'Rice_BrownSpot', 'Rice_Healthy', 'Rice_Hispa', 'Rice_LeafBlast', 'apple_apple scab', 'apple_black rot', 'apple_cedar apple rust', 'apple_healthy', 'cherry (including sour)_healthy', 'cherry (including sour)_powdery mildew', 'corn (maize)_cercospora leaf spot gray leaf spot', 'corn (maize)_common rust', 'corn (maize)_healthy', 'corn (maize)_northern leaf blight', 'grape_black rot', 'grape_esca (black measles)', 'grape_healthy', 'grape_leaf blight (isariopsis leaf spot)', 'orange_haunglongbing (citrus greening)', 'peach_bacterial spot', 'peach_healthy', 'pepper, bell_bacterial spot', 'pepper, bell_healthy', 'potato_early blight', 'potato_healthy', 'potato_late blight', 'squash_powdery mildew', 'strawberry_healthy', 'strawberry_leaf scorch', 'tomato_bacterial spot', 'tomato_early blight', 'tomato_healthy', 'tomato_late bl

Processing Test Images:   0%|          | 0/1161 [00:00<?, ?image/s]WARNING:tensorflow:5 out of the last 1165 calls to <function TensorFlowTrainer.make_predict_function.<locals>.one_step_on_data_distributed at 0x7b52fcb1c3a0> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has reduce_retracing=True option that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 324ms/step


Processing Test Images:   0%|          | 1/1161 [00:00<07:38,  2.53image/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step


Processing Test Images: 100%|█████████▉| 1159/1161 [02:24<00:00,  8.53image/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step


Processing Test Images: 100%|█████████▉| 1160/1161 [02:24<00:00,  8.00image/s]


Model Accuracy on Test Dataset (matching classes): 87.50%


# Final Test Accuracy ** 87.5%**